# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [6]:
# import libraries
import re
import nltk
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier 
from sklearn.model_selection import GridSearchCV

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    stemmed = [PorterStemmer().stem(w) for w in words]
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(MultinomialNB()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train.shape

(20972,)

In [9]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#  Train the pipeline
pipeline.fit(X_train, Y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ssifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# Test your model and report classification metrics
Y_pred = pipeline.predict(X_test)


f1_score_result = dict()
# Iterate through each column 
for i, column in enumerate(Y.columns):
    f1_score_result[column] = f1_score(Y_test[column], Y_pred[:, i], average='weighted')   #print classification report


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
print(pd.Series(f1_score_result))

related                   0.689239
request                   0.828832
offer                     0.992569
aid_related               0.756622
medical_help              0.881912
medical_products          0.923354
search_and_rescue         0.963821
security                  0.974899
military                  0.955885
child_alone               1.000000
water                     0.904112
food                      0.835949
shelter                   0.867663
clothing                  0.979168
money                     0.970351
missing_people            0.982870
refugees                  0.951357
death                     0.932731
other_aid                 0.805809
infrastructure_related    0.907188
transport                 0.931886
buildings                 0.924292
electricity               0.965240
tools                     0.990856
hospitals                 0.986861
shops                     0.993712
aid_centers               0.980876
other_infrastructure      0.936675
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Define hyperparameter grid
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],  # Adjust the range of n-grams to consider
    'tfidf__use_idf': [True, False],  # Whether to use inverse document frequency (TF-IDF)
    'clf__estimator__alpha': [0.1, 0.5, 1.0]  # Alpha parameter for Multinomial Naive Bayes
}

# Grid search for hyperparameter tuning
cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1)

cv.fit(X_train, Y_train)

# Get the best parameters
best_params = cv.best_params_

/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Test your model and report classification metrics
Y_pred = cv.predict(X_test)

f1_score_result = dict()
# Iterate through each column 
for i, column in enumerate(Y.columns):
    f1_score_result[column] = f1_score(Y_test[column], Y_pred[:, i], average='weighted') 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [ ]:
#  Train the pipeline
pipeline.fit(X_train, Y_train)

In [ ]:
# Test your model and report classification metrics
Y_pred = pipeline.predict(X_test)


f1_score_result = dict()
# Iterate through each column 
for i, column in enumerate(Y.columns):
    f1_score_result[column] = f1_score(Y_test[column], Y_pred[:, i], average='weighted')

In [ ]:
print(pd.Series(f1_score_result))

In [ ]:
# Define hyperparameter grid
parameters = {
    'clf__estimator__n_estimators': [50, 100, 200],  # Number of trees (for RandomForest)
    'clf__estimator__max_depth': [None, 10, 20],  # Maximum depth of trees (for RandomForest)
    'clf__estimator__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'clf__estimator__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}

# Grid search for hyperparameter tuning
cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1)

cv.fit(X_train, Y_train)

# Get the best parameters
best_params = cv.best_params_

In [ ]:
# Test your model and report classification metrics
Y_pred = cv.best_estima.predict(X_test)


f1_score_result = dict()
# Iterate through each column 
for i, column in enumerate(Y.columns):
    f1_score_result[column] = f1_score(Y_test[column], Y_pred[:, i], average='weighted')

In [ ]:
# Step 2: Create the machine learning pipeline with XGBoost
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(XGBClassifier()))  # XGBoost classifier
])

In [ ]:
#  Train the pipeline
pipeline.fit(X_train, Y_train)

In [ ]:
# Define the hyperparameter grid for grid search
param_grid = {
    'clf__estimator__n_estimators': [100, 200, 300],  # Number of boosting rounds
    'clf__estimator__learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'clf__estimator__max_depth': [3, 4, 5],  # Maximum depth of trees
}

In [ ]:
# Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, Y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:")
print(best_params)


In [ ]:
# Test your model with the best hyperparameters and report classification metrics
best_model = grid_search.best_estimator_
Y_pred = best_model.predict(X_test)

In [ ]:
f1_score_result = dict()
# Iterate through each column 
for i, column in enumerate(Y.columns):
    f1_score_result[column] = f1_score(Y_test[column], Y_pred[:, i], average='weighted')

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.